In [124]:

!pip install beautifulsoup4
!pip install requests


# part 1 -  scrape atleast 20 pages of product listing pages Items to scrape Product URL , Product Name, Product Price ,Rating, Number of reviews

In [126]:
import csv
import requests
from bs4 import BeautifulSoup

def scrape_page(page_url):
    response = requests.get(page_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    products = soup.find_all('div', {'data-component-type': 's-search-result'})

    product_data = []  # List to store product data

    for product in products:
        product_url_element = product.find('a', {'class': 'a-link-normal s-no-outline'})
        product_name_element = product.find('span', {'class': 'a-size-medium a-color-base a-text-normal'})
        product_price_element = product.find('span', {'class': 'a-offscreen'})
        product_rating_element = product.find('span', {'class': 'a-icon-alt'})
        product_reviews_element = product.find('span', {'class': 'a-size-base'})

        # Check if the elements exist before accessing their text attribute
        if product_url_element:
            product_url = "amazon.in" + product_url_element['href']
        else:
            product_url = "Not available"

        if product_name_element:
            product_name = product_name_element.text
        else:
            product_name = "Not available"

        if product_price_element:
            product_price = product_price_element.text
        else:
            product_price = "Not available"

        if product_rating_element:
            product_rating = product_rating_element.text
        else:
            product_rating = "Not available"

        if product_reviews_element:
            product_reviews = product_reviews_element.text
        else:
            product_reviews = "Not available"

        # Append the product data to the list
        product_data.append([product_url, product_name, product_price, product_rating, product_reviews])

    return product_data

base_url = "https://www.amazon.in/s?k=bags&crid=2M096C61O4MLT&qid=1653308124&sprefix=ba%2Caps%2C283&ref=sr_pg_"
num_pages = 20

all_product_data = []

for page in range(1, num_pages + 1):
    page_url = base_url + str(page)
    product_data = scrape_page(page_url)
    all_product_data.extend(product_data)

# Save the product data to a CSV file
csv_file = 'finalproducts.csv'
header = ['URL', 'Name', 'Price', 'Rating', 'Reviews']

with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(all_product_data)

print("Data saved to", csv_file)

   

Data saved to finalproducts.csv


# part 2 - With the Product URL received in the above case, hit each URL, and add below items: Description, ASIN, Product Description, Manufacturer


In [ ]:
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd

# Set up Chrome WebDriver
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode, without opening a browser window
service = Service("/Users/vridhikumar/Desktop/chromedriver_mac64\ 2/chromedriver")  # Replace with the path to your chromedriver executable
driver = webdriver.Chrome(service=service, options=chrome_options)

# Create an empty DataFrame
df = pd.DataFrame(columns=['Title', 'Description', 'ASIN', 'Manufacturer', 'URL'])

# Function to scrape the product page
def scrape_product_page(url):
    driver.get(url)

    # Get the page source after JavaScript execution
    page_source = driver.page_source

    # Create BeautifulSoup object for parsing
    soup = BeautifulSoup(page_source, 'html.parser')

    # Extract the title from the product page
    title_element = soup.find('span', {'id': 'productTitle'})
    title = title_element.get_text(strip=True) if title_element else None

    # Extract the description from the product page
    description_elem = soup.find('div', {'id': 'productDescription'})
    description = description_elem.get_text(strip=True) if description_elem else 'N/A'
    
    # Extract the ASIN and Manufacturer from the product page
    asin = None
    manufacturer = None
    div_element = soup.find('div', {'id': 'detailBullets_feature_div'})
    if div_element:
        li_elements = div_element.find_all('li')
        if len(li_elements) > 3:
            asin_element = li_elements[3].find('span', class_=False)
            asin = asin_element.text if asin_element else None
            manufacturer_element = li_elements[2].find('span', class_=False)
            manufacturer = manufacturer_element.text if manufacturer_element else None

    # Print or store the extracted information
    row = {'Title': title, 'Description': description, 'ASIN': asin, 'Manufacturer': manufacturer, 'URL': url}
    df.loc[len(df)] = row
    
    print("Title:", title)
    print("Description:", description)
    print("ASIN:", asin)
    print("Manufacturer:", manufacturer)
    print("URL:", url)
    print()

# URL of the search page
search_url = "https://www.amazon.in/s?k=bags&crid=2M096C61O4MLT&qid=1653308124&sprefix=ba%2Caps%2C283&ref=sr_pg_1"

# Send a GET request to the search page and retrieve the HTML content
response = requests.get(search_url)
html_content = response.text

# Create BeautifulSoup object for parsing the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Find all the product elements on the page
product_elements = soup.find_all('div', {'data-component-type': 's-search-result'})

# Extract the product URLs
product_urls = []
for element in product_elements:
    link_element = element.find('a', {'class': 'a-link-normal s-no-outline'})
    if link_element:
        product_url = "https://www.amazon.in" + link_element['href']
        product_urls.append(product_url)

# Scrape each product page
for url in product_urls:
    scrape_product_page(url)

# Close the browser and WebDriver
driver.quit()

# Print the final DataFrame
print(df)

# Save DataFrame as CSV file
df.to_csv('product_details.csv', index=False)
